In [99]:
import csv
import random
import math
import pandas as pd

In [100]:
def loadcsv(filename):
    dataset=pd.read_csv(filename)
    n=len(dataset['Pregnancies'].values)
    dataframe=[]
    for i in range (n):
        dataframe.append(dataset.iloc[i].values.tolist())
        
    return dataframe

In [101]:
def splitdataset(dataset, splitratio):
    #67% training size
	trainsize = int(len(dataset) * splitratio);
	trainset = []
	copy = list(dataset);    
	while len(trainset) < trainsize:
#generate indices for the dataset list randomly to pick ele for training data
		index = random.randrange(len(copy));       
		trainset.append(copy.pop(index))    
	return [trainset, copy]

In [102]:
def separatebyclass(dataset):
	separated = {}
	for i in range(len(dataset)):
		vector = dataset[i]
		if (vector[-1] not in separated):
			separated[vector[-1]] = []
		separated[vector[-1]].append(vector)
	return separated

In [103]:
def mean(numbers):
	return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)

In [104]:
def summarize(dataset): #creates a dictionary of classes
	summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)];
	del summaries[-1]#excluding labels +ve or -ve
	print(summaries[-1])
	return summaries

In [105]:
def summarizebyclass(dataset):
	separated = separatebyclass(dataset); 
#     print(separated)
	summaries = {}
	for classvalue, instances in separated.items():       
		summaries[classvalue] = summarize(instances) #summarize is used to cal to mean and std
	return summaries

In [106]:
def calculateprobability(x, mean, stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [107]:
def calculateclassprobabilities(summaries, inputvector):
	probabilities = {} # probabilities contains the all prob of all class of test data
	for classvalue, classsummaries in summaries.items():#class and attribute information as mean and sd
		probabilities[classvalue] = 1
		for i in range(len(classsummaries)):
			mean, stdev = classsummaries[i] #take mean and sd of every attribute for class 0 and 1 seperaely
			x = inputvector[i] #testvector's first attribute
			probabilities[classvalue] *= calculateprobability(x, mean, stdev);#use normal dist
	return probabilities

In [108]:
def predict(summaries, inputvector): #training and test data is passed
	probabilities = calculateclassprobabilities(summaries, inputvector)
# 	print(probabilities)   
	bestLabel, bestProb = None, -1
	for classvalue, probability in probabilities.items():#assigns that class which has he highest prob
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classvalue
	return bestLabel

In [109]:
def getpredictions(summaries, testset):
	predictions = []
	for i in range(len(testset)):
		result = predict(summaries, testset[i])
		predictions.append(result)
# 		print(result)
	return predictions

In [110]:
def getaccuracy(testset, predictions):
	correct = 0
	for i in range(len(testset)):
		if testset[i][-1] == predictions[i]:
			correct += 1
	return (correct/float(len(testset))) * 100.0

In [111]:
def main():
	filename = 'bayes.csv'
	splitratio = 0.67
	dataset = loadcsv(filename);
     
	trainingset, testset = splitdataset(dataset, splitratio)
	print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingset), len(testset)))
	# prepare model
	summaries = summarizebyclass(trainingset);    
	#print(summaries)
    # test model
	predictions = getpredictions(summaries, testset) #find the predictions of test data with the training data
	accuracy = getaccuracy(testset, predictions)
	print('Accuracy of the classifier is : {0}%'.format(accuracy))

In [112]:
main()


Split 767 rows into train=513 and test=254 rows
(37.30107526881721, 10.837657018394614)
(31.38532110091743, 11.32474481914113)
Accuracy of the classifier is : 76.37795275590551%
